# Overview
this is a training script for the RNN. To train against multiple types of agents, edit the run_game method.
In order for this program to work record_data must be working

In [1]:
import torch
import pickle
from uncertainty_rnn import BoardGuesserNet
import torch.optim as optim
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from collections import deque
import sys
from modified_play_game import play_game


ModuleNotFoundError: No module named 'chess'

# Board Guess

In [2]:
# set up Board Guesser Net
guessNet = BoardGuesserNet()

#set up loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(guessNet.parameters(), lr = .001)

# Constants
train_iterations = 100000
validation_count  = 50 # after every 20 games check validation score
load_in_weights = True

C:\Users\Michael Jurado\workspace\ReconChess\Another_Chess_Bot\uncertainty_rnn.py:12: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.conv1.weight, gain = 1)
C:\Users\Michael Jurado\workspace\ReconChess\Another_Chess_Bot\uncertainty_rnn.py:17: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.conv2.weight, gain = 1)
C:\Users\Michael Jurado\workspace\ReconChess\Another_Chess_Bot\uncertainty_rnn.py:27: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.dense1.weight, gain = 1)


# Definer helpers

In [3]:

def save_model(guessNet):
    # definer helpers
    try:
        os.mkdir("rnn_model")
    except Exception as e:
        print(e)

    torch.save(guessNet.state_dict(), "rnn_model")
    
def load_model(guessNet):
    guessNet.load_state_dict(torch.load("rnn_model"))
    
def run_game():
    """
    Returns X_train_batch, y_train_batch, which are both numpy arrays
    
    NOTE: Addit command to change which agent plays. (Must save data in specified format though)
    """
    start = time.time()
    white_data, black_data = play_game("random_agent_save_obs", "random_agent_save_obs")
    

    player_sense_list, player_truth_board_list = white_data
    
    X_train_batch = np.array(player_sense_list)
    y_train_batch = np.array(player_truth_board_list)
    return X_train_batch, y_train_batch


def record_game_loss(pred_list, y_train_batch):
    """
    Record mse between truth board and predictions. 
    THis is for validation
    """
    if isinstnace(pred_list, torch.Tensor):
        y_train_batch = y_train_batch.detach().cpu().numpy()
        pred_list = pred_list.detach().cpu().numpy()
    return np.sum((pred_list - y_train_batch)**2)



    
def create_loss_plot(train_loss_history):
    plt.figure()

    x_axis = (np.arange(len(train_loss_history)))
    
    plt.plot(x_axis, train_loss_history, label = "train_loss")
    
    plt.legend(loc="upper left")
    plt.ylabel("MSE")
    plt.xlabel("epochs")    
    try:
        plt.savefig("rnn_loss.png")
    except Exception as e:
        print("Permission denied saving loss")
        
    plt.cla()
    plt.clf()
    plt.close()
    
    

# Training Step

In [9]:
if load_in_weights:
    load_model(guessNet)
    # TODO

# contains train and test lost history
train_loss_history = []
test_loss_history = []
plt.figure()

train_loss_queue = deque(maxlen = 100)  # holds a temporary queue of train lost
smallestLoss = sys.maxsize
lastImprovement = 0
for epoch in range(train_iterations):
    X_train_batch, y_train_batch = run_game() # plays a game random versus random

    X_train_batch = torch.Tensor(X_train_batch)
    # training step
    pred_labels = guessNet(X_train_batch)
    y_train_batch = torch.Tensor(y_train_batch)
    loss = criterion(pred_labels, y_train_batch)

    loss.backward()
    optimizer.step() # magic    
    train_loss_queue.append(loss.detach().cpu().numpy())
    
    loss = np.mean(train_loss_queue)
    train_loss_history.append(np.mean(train_loss_queue))

    if (epoch +1) % 5 == 0:  # save loss plot every 5 steps
        create_loss_plot(train_loss_history)
        
    # run a validation every 100 steps
    if (epoch+ 1) % 100 == 0:
        print("validation time")
        total_val_loss = 0
        for i in range(validation_count):
            X_train_batch, y_train_batch = run_game() # plays a game random versus random
            X_train_batch = torch.Tensor(X_train_batch)
            
            # training step
            pred_labels = guessNet(X_train_batch)
            y_train_batch = torch.Tensor(y_train_batch)
            total_val_loss += criterion(pred_labels, y_train_batch)   
        val_loss = total_val_loss/ validation_count
        
        if val_loss < smallestLoss:
            print("better model found with loss", val_loss)
            save_model(guessNet)
            smallestLoss =  val_loss
    
    # decay learning rate
    if (epoch+1) % 200 == 0:
        for param_group in optimizer.param_groups:
                param_group['lr'] *= .98     
                param_group['lr'] = max(.0001, param_group['lr'])
        print("decaying learning rate to", param_group["lr"])
        
    # print out pred_board[0]
    if (epoch+1) % 100 == 0:
        print("pred board for first time step")
        index = 0
        for channel in [np.argwhere(i > .5) for i in pred_labels[0].detach().numpy()]:
            print("active arguments in channel", index)
            print(channel)
            index += 1
    
        

            

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>